# OWSLib OGC Features API

This notebook provides a quick introduction to using the [OGC Features API]() with the help of the OWSLib Python library. We'll connect to the [pygeoapi](https://live.osgeo.org/en/overview/pygeoapi_overview.html) demo services installed on OSGeoLive.

Before running the notebook, ensure that the pygeoapi service is running. On the OSGeoLive Application Menu go to `Geospatial > Web Services > pygeoapi > Start pygeoapi`. Then check in a browser that the pygeoapi homepage is running at http://localhost:5000/.

We will start by importing `owslib` and checking its version. This notebook has been tested with version `0.33.0`.
As the new OGC APIs return JSON rather than XML (as with the older WxS services such as WFS) we will also import the the standard Python [json](https://docs.python.org/3.13/library/json.html) library to easily print and access data in JSON structures. 

Reading the OGC Features API can be easily done using the [requests](https://pypi.org/project/requests/) library (which OWSLib uses underneath), but using OWSLib helps with building more complex URLs, such as using [CQL](https://portal.ogc.org/files/96288) queries, adding and deleting features, handling authentication, and logging. 

In [ ]:
import owslib
print(owslib.__version__)
import json

pygeoapi supports multiple OGC APIs. Let's see what APIs are available on the pygeoapi demo. We can "pretty-print" these services using `json.dumps`, and setting the `indent` parameter. In this notebook we will be connecting to the `ogcapi-features` API. 

In [ ]:
from owslib.ogcapi.features import Features
service = Features("http://localhost:5000")
print(json.dumps(service.conformance(), indent=4))

Different feature types are stored in collections. Let's explore what feature collections are available on the server.

In [ ]:
# get the names of feature collections
feature_collections = service.feature_collections()
print(feature_collections)

There is only a single 'lakes' feature collection on the demo server. Let's see how many features the collection contains. 

In [ ]:
lake_features = service.collection_items('lakes')

# print(json.dumps(lake_features, indent=4))
print(f'Features returned in response: {len(lake_features['features'])}')

# as paging is implemented only 10 features are returned in the collection
print(f'Number returned: {lake_features['numberReturned']}')

# we can see the full count using numberMatched
print(f'Number matched: {lake_features['numberMatched']}')

Now we have connected to a service and accessed features as GeoJSON we can render them using [GeoPandas](https://geopandas.org/en/stable/). 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

gdf = gpd.GeoDataFrame.from_features(
    lake_features["features"],  # pass the 'features' list
)

first_feature = gdf.iloc[[0]]  # use [[0]] to keep it as a GeoDataFrame
first_feature.plot()
plt.title(f"{first_feature['name'].values[0]}")  # set the title to the lake name
plt.show()

We can setup logging in OWSLib to capture request URLs and parameters. We'll do that below, and then get a list of all attributes in the 'lakes' collection using `collection_queryables`. 

In [ ]:
import logging
owslib_logger = logging.getLogger('owslib')
owslib_logger.setLevel(logging.DEBUG)
stream_handler = logging.StreamHandler()

# add the handler to the logger (avoid adding multiple handlers accidentally)
if not any(isinstance(h, logging.StreamHandler) for h in owslib_logger.handlers):
    owslib_logger.addHandler(stream_handler)
    
print(json.dumps(service.collection_queryables('lakes'), indent=4))

Finally we'll query the `name` attribute to return a single lake feature.

In [ ]:
lake_victoria = service.collection_items('lakes', name='Lake Victoria')

gdf = gpd.GeoDataFrame.from_features(
    lake_victoria["features"]
)

gdf.iloc[[0]].plot()
plt.title("Lake Victoria")
plt.show()

For further details please see the [OWSLib Documentation](https://owslib.readthedocs.io/en/latest/usage.html#ogc-api). 